In [1]:
# general
import datetime as dt
import os
# data 
import xarray as xr 
import numpy as np
import pandas as pd
# plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
# helper tools
from utils.helper_funcs import create_windrose_df
import utils.helper_funcs as hf
from metpy import calc, units
# make plotly work 
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
# import data
sublimation_30min_df = pd.read_csv('../01_data/processed_data/sublimation/sublimation_met_timeseries_30T.csv', index_col=0, parse_dates=True)

In [3]:
sublimation_30min_df['sublimation'] = sublimation_30min_df['sublimation'].where(sublimation_30min_df['precipitation'] < 0.1, np.nan)

In [4]:
def add_path(fig, path="/home/dlhogan/GitHub/Synoptic-Sublimation/03_results/blowing_snow_and_sublimation.ipynb"):
    # annotate the upper right corner with the script that created the plot
    fig.add_annotation(
        text='Produced using: {}'.format(path),
        xref='paper', x=1, y=1,
        yref='paper',
        showarrow=False,
        font=dict(size=8),
        xanchor='right',
        yanchor='bottom',
        align='right'
    )
    return fig

In [5]:

# create a scatter plot with the wind speed on the x-axis and the sublimation rate on the y-axis and the markers sized by the blowing snow flux (log scale)
fig1 = px.scatter(sublimation_30min_df.dropna(), 
                 x=f'wind_speed', 
                 y=f'sublimation', 
                 size='bsf', 
                 # color by wind direction
                 color=f'temperature',
                 color_continuous_scale='RdBu_r',
                 range_color=[-10,0],
                 # make edge color black
                 # provide size scale
                    size_max=70,
                 title=f'Wind Speed and Sublimation Rate <br> sized by BSF')


fig1.update_traces(marker=dict(
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                    text = sublimation_30min_df.dropna().index,
                    hovertemplate='Date: %{text} &deg;<br>Wind Speed: %{x} m/s<br>Sublimation Rate: %{y} mm/hr<br>Temperature: %{marker.color:.2f} &deg;C',
                  selector=dict(mode='markers'))

df_l = sublimation_30min_df.sort_values('bsf')
# fill df_1 with 0 for nan values
df_l = df_l.fillna(0)

fig2 = px.scatter(
    df_l,
    x=np.zeros(5),
    y=np.array(["0","1","5","10","50+",]),
    size=np.array([0.01,1,5,10,50]),
    size_max=40,
    # set color to gray
)

fig2.update_traces(marker=dict(
                            color='gray',
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))



fig = go.Figure(
    data=[t for t in fig1.data] + [t.update(xaxis="x2", yaxis="y2") for t in fig2.data],
    layout=fig1.layout,
)
# annotate path
add_path(fig)
# now config axes appropriately
fig.update_layout(
    xaxis_domain=[0, 0.90],
    xaxis1=dict(title='Wind Speed (m/s)',
                ),
    yaxis1=dict(title='Sublimation Rate (mm/hr)',
                range=[-0.05, 0.2]),
    xaxis2={"domain": [0.905, 1], "matches": None, "visible": False},
    yaxis2={"title":"Blowing Snow Flux (g/m^2/s)","anchor": "free", "overlaying": "y", "side": "right", "position": 1},
    showlegend=False,
    height=600, width=1000,
    coloraxis_colorbar_title_text ='Temperature (&deg;C)'
)

# size the legend in a log scale
fig.update_layout(legend=dict(itemsizing='constant',))
# move the colorbar axis to the right, flip orientation
fig.update_layout(coloraxis_colorbar=dict(yanchor="top", 
                                          y=-0.2, 
                                          x=-0.10, 
                                          xanchor="left", 
                                          title='Temperature (&deg;C)',
                                          orientation='h'))
save_fig = True
if save_fig:
    fig.write_image('../04_products/figures/draft/wind_speed_sublimation_bsf_temperature.png')

In [ ]:

# create a scatter plot with the wind speed on the x-axis and the sublimation rate on the y-axis and the markers sized by the blowing snow flux (log scale)
fig1 = px.scatter(sublimation_30min_df.dropna(), 
                 x=f'wind_speed', 
                 y=f'sublimation', 
                 size='bsf', 
                 # color by wind direction
                 color=f'wind_direction',
                 color_continuous_scale='twilight',
                #  range_color=[0,360],
                 # make edge color black
                 # provide size scale
                 size_max=70,
                 title=f'')


fig1.update_traces(marker=dict(
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                    text = sublimation_30min_df.dropna().index,
                    hovertemplate='Date: %{text} &deg;<br>Wind Speed: %{x} m/s<br>Sublimation Rate: %{y} mm/hr<br>Wind Direction: %{marker.color:.2f} &deg;C',
                  selector=dict(mode='markers'))

df_l = sublimation_30min_df.sort_values('bsf')
# fill df_1 with 0 for nan values
df_l = df_l.fillna(0)

fig2 = px.scatter(
    df_l,
    x=np.zeros(5),
    y=np.array(["0","1","5","10","50+",]),
    size=np.array([0.01,1,5,10,50]),
    size_max=40,
    # set color to gray
)

fig2.update_traces(marker=dict(
                            color='gray',
                              line=dict(width=1,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))



fig = go.Figure(
    data=[t for t in fig1.data] + [t.update(xaxis="x2", yaxis="y2") for t in fig2.data],
    layout=fig1.layout,
)
add_path(fig)
# now config axes appropriately
fig.update_layout(
    xaxis_domain=[0, 0.90],
    xaxis1=dict(title='Wind Speed (m/s)',
                ),
    yaxis1=dict(title='Sublimation Rate (mm/hr)',
                range=[-0.05, 0.2]),
    xaxis2={"domain": [0.905, 1], "matches": None, "visible": False},
    yaxis2={"title":"Blowing Snow Flux (g/m^2/s)","anchor": "free", "overlaying": "y", "side": "right", "position": 1},
    showlegend=False,
    height=600, width=1000,
    coloraxis_colorbar_title_text ='Wind Direction (&deg;)'
)

# size the legend in a log scale
fig.update_layout(legend=dict(itemsizing='constant',))
# move the colorbar axis to the right, flip orientation
fig.update_layout(coloraxis_colorbar=dict(yanchor="top", 
                                          y=-0.2, 
                                          x=-0.10, 
                                          xanchor="left", 
                                          title='Wind Direction (&deg;)',
                                          orientation='h'))
save_fig = True
if save_fig:
    fig.write_image('../04_products/figures/draft/wind_speed_sublimation_bsf_wind_dir.png')